In [1]:
import os
import json

from library_dicom.dicom_processor.model.Series import Series
from library_dicom.dicom_processor.model.SeriesPT import SeriesPT
from library_dicom.dicom_processor.tools.folders import get_series_path, write_json_file, remove_bi_file
from library_dicom.dicom_processor.tools.series import get_series_object


In [2]:
batch_path = []
for i in range(0,80):
    batch_path.append('/media/salim/DD 2To/AHL2011/AHLGroupe'+str(i))

In [3]:
series_paths = []
for input_path in batch_path:
    series_paths_current = get_series_path(input_path)
    series_paths = series_paths + series_paths_current
export_folder = '/media/salim/DD 2To/Json Ahl'

In [4]:
not_image_folder=[]
non_valid_series=[]
non_attenuation_corrected=[]
empty_folders = []
secondary_series = []
localizer_series = []

for serie_path in series_paths:
    if(len(os.listdir(serie_path)) == 0):
        empty_folders.append(serie_path)
        continue
    try:    
        dicom_serie = get_series_object(serie_path)
        dicomsInfo = dicom_serie.get_series_details()
        json_string = json.dumps(dicomsInfo)
        remove_bi_file(serie_path)
        if(dicom_serie.is_image_modality() == False):
            not_image_folder.append(serie_path)
        if(dicom_serie.is_series_valid() == False):
            non_valid_series.append(serie_path)
        if(type(dicom_serie) == SeriesPT and dicom_serie.is_corrected_attenuation() == False):
            non_attenuation_corrected.append(serie_path)
        if(dicom_serie.is_primary_image() == False):
            secondary_series.append(serie_path)
        if(dicom_serie.is_localizer_series() == True):
            localizer_series.append(serie_path)
    except Exception as err:
        print(err)
        print(serie_path)
        print(dicomsInfo)
    
    write_json_file(export_folder, dicomsInfo['series']['SeriesInstanceUID'], json_string)
    

wrong number of slice
wrong number of slice
wrong number of slice
wrong number of slice
wrong number of slice
wrong number of slice


In [5]:
#print(not_image_folder)
#print(non_valid_series)
#print(non_attenuation_corrected)
#print(empty_folders)
#print(secondary_series)
print(localizer_series)

['/media/salim/DD 2To/AHL2011/AHLGroupe2/Batch1/13011101021020/PET0/1.3.12.2.1107.5.1.4.1003.30000013060304544625000004371', '/media/salim/DD 2To/AHL2011/AHLGroupe3/Batch1/13011101031005/PET0/1.3.12.2.1107.5.1.4.1017.30000012022406274515600000042']


In [ ]:
import shutil
for image_folder in not_image_folder:
    shutil.rmtree(image_folder)

for empty_folder in empty_folders:
    shutil.rmtree(empty_folder)

for non_attenuation in non_attenuation_corrected:
    shutil.rmtree(non_attenuation)